# PATH 설정

## Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Package

In [2]:
import os
import yaml
import numpy as np
from tqdm import tqdm
from itertools import product

import torch.cuda
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

# Config

In [3]:
import yaml

In [4]:
with open('.config.yaml', 'r') as f:
    args = yaml.safe_load(f)
args

{'num_classes': 21,
 'hidden_layer': 2,
 'batch_size': 16,
 'lr': 0.01,
 'weight_decay': 0.9,
 'epochs': 50,
 'raw_size': 1024,
 'post_size': 224,
 'train_size': 1500,
 'val_size': 124,
 'test_size': 124}

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
args['device'] = device
print(torch.cuda.get_device_name())

Tesla V100-SXM2-16GB


# Setup

## Model

In [6]:
from src.model import load_mobilenet_large

model = load_mobilenet_large(args['num_classes'])
# model

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth


  0%|          | 0.00/74.2M [00:00<?, ?B/s]

## Optimizer

In [7]:
optimizer = optim.SGD(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

## Data

In [8]:
from src.preprocessing import load_trans

trans = load_trans()
trans

Compose(
    Resize(size=(100, 100), interpolation=bilinear, max_size=None, antialias=None)
    ToTensor()
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
)

In [9]:
import torchvision
# train_dataset = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/data/training/processed_train')
# train_dataset = torch.load('/content/drive/MyDrive/data/training/processed_train/OBJ00013_PS3_K3_NIA0078_img')
# torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, **kwargs, drop_last = True if train else False)

class objectdataset(torch.utils.data.Dataset) :
    def __init__(self, root, transforms = None) :
        self.root = root
        self.lists = list(sorted(os.listdir(root)))
        self.transforms = transforms
        self.images = [file for file in self.lists if file.endswith("_img")]
        self.targets = [file for file in self.lists if file.endswith("_target")]

    def __getitem__(self, idx) : 
        img_path = os.path.join(self.root, self.images[idx])
        target_path = os.path.join(self.root, self.targets[idx])
        if(self.transforms == None) :
            images = torch.load(img_path)
            targets = torch.load(target_path)
            id = torch.tensor([idx])

        ##안돌아간다면 자료형, tensor형태 확인 필요

        ##transfrom이 none이 아닐 때에는 코드 추가 필요

        return images, targets

    def __len__(self) :
        return len(self.images)
        


In [10]:
train_dataset = objectdataset('/content/drive/MyDrive/data/training/processed_train', None)
# valid_dataset = objectdataset()

# Train

In [11]:
torch.backends.cudnn.benchmark = True

## Train 함수 

Data Loader에 따라 함수가 달라질 것

In [12]:
def train(model, optimizer, loader):
    print('Training')

    # initialize tqdm progress bar
    prog_bar = tqdm(loader, total=len(loader))
    loss_cls = []
    loss_box = []
    loss_obj = []
    loss_rpn_box = []
    loss = []
    for epoch in range(args['epochs']):
        for i, data in enumerate(prog_bar):
            optimizer.zero_grad()
            images, targets = data

            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            loss_value = losses.item()
            loss.append(loss_value)

            loss_cls.append(loss_dict['loss_classifier'].detach().cpu().numpy())
            loss_box.append(loss_dict['loss_box_reg'].detach().cpu().numpy())
            loss_obj.append(loss_dict['loss_objectness'].detach().cpu().numpy())
            loss_rpn_box.append(loss_dict['loss_rpn_box_reg'].detach().cpu().numpy())
            losses.backward()
            optimizer.step()
            # update the loss value beside the progress bar for each iteration
            prog_bar.set_description(desc=f"Loss: {loss_value:.4f}")
        if epoch % 5 == 0:
            torch.save(model.state_dict(), f"snapshot/model/{epoch}th_epoch.pt")
        print(f"Epoch {epoch+1} is DONE")
    return loss, loss_cls, loss_box, loss_obj, loss_rpn_box

In [ ]:
losses = train(model.to(device), optimizer, train_dataset)


Training


Loss: 0.6232: 100%|██████████| 1748/1748 [03:09<00:00,  9.22it/s]


Epoch 1 is DONE
Epoch 2 is DONE
Epoch 3 is DONE
Epoch 4 is DONE
Epoch 5 is DONE
Epoch 6 is DONE
Epoch 7 is DONE
Epoch 8 is DONE
Epoch 9 is DONE
Epoch 10 is DONE
Epoch 11 is DONE
Epoch 12 is DONE
Epoch 13 is DONE
Epoch 14 is DONE
Epoch 15 is DONE
Epoch 16 is DONE
Epoch 17 is DONE
Epoch 18 is DONE
Epoch 19 is DONE
Epoch 20 is DONE
Epoch 21 is DONE
Epoch 22 is DONE
Epoch 23 is DONE
Epoch 24 is DONE
Epoch 25 is DONE
Epoch 26 is DONE
Epoch 27 is DONE
Epoch 28 is DONE


# Loss

In [ ]:
loss_fc = torch.nn.CrossEntropyLoss()

# Save

In [ ]:
np.savez(f"snapshot/loss/train_loss_{model_type}_{args['lr']}.npz", total=np.array(losses[0]), cls=np.array(losses[1]), box=np.array(losses[2])
          , objectness=np.array(losses[3]), rpn_box=np.array(losses[4]))

In [ ]:
import torch

a = torch.load('/content/drive/MyDrive/data/training/processed_train/OBJ00013_PS3_K3_NIA0078_img')
a

(tensor([[[-0.4667, -0.3882, -0.4510,  ..., -0.9059, -0.8980, -0.7412],
          [-0.4118, -0.4353, -0.5137,  ..., -0.8118, -0.8980, -0.9216],
          [-0.4902, -0.4667, -0.5294,  ..., -0.7490, -0.8588, -0.8824],
          ...,
          [-0.7490, -0.7647, -0.7412,  ..., -0.2627, -0.3412, -0.3725],
          [-0.6549, -0.7569, -0.7569,  ..., -0.1922, -0.3412, -0.3725],
          [-0.5765, -0.7098, -0.7647,  ..., -0.1216, -0.2863, -0.3804]],
 
         [[-0.5529, -0.4980, -0.5216,  ..., -0.8902, -0.8431, -0.6471],
          [-0.4980, -0.5216, -0.5686,  ..., -0.8745, -0.8824, -0.8745],
          [-0.5686, -0.5294, -0.5765,  ..., -0.8118, -0.9137, -0.8980],
          ...,
          [-0.7333, -0.7176, -0.7098,  ..., -0.1922, -0.2706, -0.3098],
          [-0.6392, -0.7255, -0.7255,  ..., -0.1216, -0.2627, -0.3020],
          [-0.5686, -0.7098, -0.7255,  ..., -0.0431, -0.2157, -0.3098]],
 
         [[-0.5843, -0.5922, -0.6314,  ..., -0.9294, -0.9137, -0.7255],
          [-0.5608, -0.6314,

In [ ]:
a[0][0]

tensor([[-0.4667, -0.3882, -0.4510,  ..., -0.9059, -0.8980, -0.7412],
        [-0.4118, -0.4353, -0.5137,  ..., -0.8118, -0.8980, -0.9216],
        [-0.4902, -0.4667, -0.5294,  ..., -0.7490, -0.8588, -0.8824],
        ...,
        [-0.7490, -0.7647, -0.7412,  ..., -0.2627, -0.3412, -0.3725],
        [-0.6549, -0.7569, -0.7569,  ..., -0.1922, -0.3412, -0.3725],
        [-0.5765, -0.7098, -0.7647,  ..., -0.1216, -0.2863, -0.3804]])

In [ ]:
a[0][1]

tensor([[-0.5529, -0.4980, -0.5216,  ..., -0.8902, -0.8431, -0.6471],
        [-0.4980, -0.5216, -0.5686,  ..., -0.8745, -0.8824, -0.8745],
        [-0.5686, -0.5294, -0.5765,  ..., -0.8118, -0.9137, -0.8980],
        ...,
        [-0.7333, -0.7176, -0.7098,  ..., -0.1922, -0.2706, -0.3098],
        [-0.6392, -0.7255, -0.7255,  ..., -0.1216, -0.2627, -0.3020],
        [-0.5686, -0.7098, -0.7255,  ..., -0.0431, -0.2157, -0.3098]])